# IMDB Sentiment Classifier
### Using Hugging Face with the SageMaker SDK

# What We're Going To Do:

#### Installation
1. Install the SageMaker SDK and the Hugging Face libraries
1. Start a SageMaker session, including the default IAM role and S3 bucket
    
#### Data Preparation
1. Tokenization: Download and prepare our IMDB dataset for NLP model training
1. Upload our tokenized and split dataset to S3

#### Model Training
1. Setup an Estimator
1. Train a model

#### Realtime Inference
1. Prepare the model for deployment
1. Deploy the model and create a Predictor
1. Make inferences using a Predictor

#### Clean Up

---
# But What _Is_ Machine Learning?

For our purposes, we can think of machine learning as a method of using computers to learn the rules of computation. 

For example, in a traditional computation like adding two integers, we supply the input data, the integers 2 and 3, and wish to apply a rule, addition, to compute the output, 5. Computers are convenient for these types of operations for obvious historical reasons.

However, with machine learning, we supply the input and output data, but are interested in computing the unknown rules that generated our output from the input. This process is not magic. Behind the scenes, machine learning relies on statistical techniques and often complex framing of the problem as one of optimizing the fit of rules that minimize the error between the input and output data. Both how this optimization problem is framed and what particular mechanisms are employed to use computers to fit optimized rules to the data is at the frontier of machine learning research. 

Due to the increasingly convenient and economical benefits of cloud computing of the past decade, machine learning has become more accessible and democratized. However, to perform machine learning in a cloud environment, one is still responsible for the data preparation, training, and inference infrastructure. This is where Amazon SageMaker is beneficial. It's a machine learning service that you can use to build, train, and deploy machine learning models for virtually any use case.

![diagram](assets/what-is-ml.svg)

---
# Installation
##### ⏰ About 30 seconds

This section has nothing to do with machine learning, but sets up our development environment with the requisite SDKs and AWS constructs we'll need to perform machine learning. In particular, we'll fix specific versions of the SageMaker and Hugging Face SDKs, as well as direct our SageMaker Studio session to use a particular S3 bucket for staging our input and output data.

In [19]:
%%time
%%capture

import os

DATASETS_VERSION = "1.6.2"
TRANSFORMERS_VERSION = "4.5.0"
SAGEMAKER_VERSION = "2.40.0"

requirements_txt = f"""numpy
pandas
transformers=={TRANSFORMERS_VERSION}
datasets=={DATASETS_VERSION}
"""

with open(os.path.join(os.getcwd(), "scripts", "requirements.txt"), "w") as f:
    f.write(requirements_txt)

!pip install --upgrade "sagemaker==$SAGEMAKER_VERSION" "transformers==$TRANSFORMERS_VERSION" "datasets[s3]==$DATASETS_VERSION"
!conda install -c conda-forge ipywidgets -y

# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)

CPU times: user 34.2 ms, sys: 47.4 ms, total: 81.6 ms
Wall time: 11.2 s


In [20]:
%%capture

import boto3
import botocore
import sagemaker
import sagemaker.huggingface

session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()

In [21]:
print(f"SageMaker role arn: {role}")
print(f"SageMaker bucket: {session.default_bucket()}")
print(f"SageMaker session region: {session.boto_region_name}")

SageMaker role arn: arn:aws:iam::934284400219:role/service-role/AmazonSageMaker-ExecutionRole-20210510T080103
SageMaker bucket: sagemaker-us-east-1-934284400219
SageMaker session region: us-east-1


---
# Data Preparation

### Download and Split the Dataset

Machine learning datasets are often a mixture of labeled and unlabeled data. For this example, we'll only be using labeled from the IMDB movie reviews. 

When a model is trained, the process feeds labeled examples from our dataset into the training algorithm, which evaluates its performance against other labeled examples in the dataset. If the model is doing well, then the error between its predictions and the test data will be low. But we need to first decide how much of our dataset will be used for training and how much will be used for evaluating the model as it is trained. For our example, we'll simply split the labeled dataset in half and use one half for training and the other for testing.

In [23]:
%% time

import importlib
import pandas
import datasets
from transformers import AutoTokenizer
from sagemaker_demo_helper import SageMakerDemoHelper

model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset, test_dataset = datasets.load_dataset(
    "imdb", 
    ignore_verifications = True,
    split = ["train", "test"]
)

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b)


### Tokenization
##### ⏰ About 1 minute

NLP models are not trained directly against the natural languages they form predictions over. Generally speaking, machine learning models are trained with numerical inputs. Tokenization is the data preparation process by which we take our natural English language movie reviews and transform them into numbers the model training algorithm understands. There are many different ways to tokenize natural language data. In our case we will select the tokenizer that was originally used for training the pretrained [DiltilBERT model Hugging Face provides](https://huggingface.co/distilbert-base-uncased).

In [8]:
%%time

tokenize = lambda batch: tokenizer(
    batch["text"], 
    padding = "max_length", 
    truncation = "longest_first"
)

train_ds = train_dataset.shuffle().map(tokenize)
test_ds = test_dataset.shuffle().map(tokenize)

try:
    train_ds = train_ds.rename_column("label", "labels")
    test_ds = test_ds.rename_column("label", "labels")
except:
    pass

columns = ["input_ids", "attention_mask", "labels"]
train_ds.set_format("torch", columns = columns)
test_ds.set_format("torch", columns = columns)


CPU times: user 1min 5s, sys: 584 ms, total: 1min 6s
Wall time: 1min 5s


### So What Does a Tokenized Natural Language Dataset Look Like?

In [10]:
train_ds.to_pandas().head(100)[["text", "labels", "input_ids", "attention_mask"]]

,text,labels,input_ids,attention_mask
0,"Last November, I had a chance to see this film...",1,"[101, 2197, 2281, 1010, 1045, 2018, 1037, 3382...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,It was great to see some of my favorite stars ...,0,"[101, 2009, 2001, 2307, 2000, 2156, 2070, 1997...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"Well, this might be one of the funniest movies...",1,"[101, 2092, 1010, 2023, 2453, 2022, 2028, 1997...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"Right from the start you see that ""Anchors Awe...",1,"[101, 2157, 2013, 1996, 2707, 2017, 2156, 2008...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,I watched this film in shire joy.<br /><br />T...,1,"[101, 1045, 3427, 2023, 2143, 1999, 13182, 656...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
95,The creativeness of this movie was lost from t...,0,"[101, 1996, 5541, 2791, 1997, 2023, 3185, 2001...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
96,This was an exteremely good historical drama. ...,1,"[101, 2023, 2001, 2019, 4654, 3334, 21382, 213...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
97,I was expecting a lot better from the Battlest...,0,"[101, 1045, 2001, 8074, 1037, 2843, 2488, 2013...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
98,Creative use of modern and mystical elements: ...,1,"[101, 5541, 2224, 1997, 2715, 1998, 17529, 378...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


### WTF?

- `text` contains the raw English IMDB movie reviews 
- `labels` are the sentiment values for each review where `1` is positive and `0` is negative
- `input_ids` are the tokens, referred to here as IDs. Hugging Face associates the token IDs with the raw numerical token values that are fed into the model training loop.
- `attention_mask` refers to which elements of the `input_ids` vector are actually processed in the training loop. Because each original `text` is a different length, we've chosen to pad the data to the same length. The attention mask makes sure the empty padding values are not used in the training loop.

### Upload the Dataset to S3
##### ⏰ About 5 seconds

In [12]:
%%time

from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()

s3_prefix = "datasets/imdb-binary-classification"
training_input_path = f"s3://{bucket}/{s3_prefix}/train"
test_input_path = f"s3://{bucket}/{s3_prefix}/test"

train_ds.save_to_disk(training_input_path, fs = s3)
test_ds.save_to_disk(test_input_path, fs = s3)

CPU times: user 889 ms, sys: 475 ms, total: 1.36 s
Wall time: 6.11 s


<span style="font-size: 16px;"><a href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-934284400219?region=us-east-1&prefix=datasets/imdb-binary-classification/&showversions=false">Prove it Landed in S3</a></span>

---
# Model Training

### Setup an Estimator

Estimators are part of the SageMaker SDK and represent at a high-level the model training job, data access, and managed infrastructure required to produce the trained model artifact. Using the latest version of the SageMaker SDK, we can leverage its [Hugging Face integration](https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugging-face) to simplify the training process.

How do we evaluate the model training performance as its running? When we train a model using SageMaker, we can monitor several metrics in real time in AWS using Amazon CloudWatch. In particular, we'll look at two varieties of metrics: the EC2 training instance metrics and the training algorithm metrics. The EC2 training instance metrics will be supplied by SageMaker without needing to configure anything. But to capture the specific Hugging Face model training metrics, we need to tell the `HuggingFace` estimator that we're interested in specific ones, which we do by specifiying in the `metric_definitions` list below. There are many more detailed metrics we can subscribe to, but for this example we will only pay attention to two: the epoch and the loss. 

Loosely speaking, when we train a machine learning model over a dataset, one complete run through the dataset is called an _epoch_. Usually models are trained for more than one epoch, and in our case we will train for three epochs. The _loss_ is a generalized notion of the error associated with the model's performance against the test dataset we split from the training set at the beginning of this notebook. The lower the loss is, the better our model is at predicting correct sentiment labels on the test dataset, which it has never seen before.

In [13]:
from sagemaker.huggingface import HuggingFace

job_name = "imdb-huggingface"

metric_definitions = [
    { "Name": "epoch", "Regex": "'epoch': ([0-9]+(.|e\-)[0-9]+),?" },
    { "Name": "loss", "Regex": "'loss': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "learning_rate", "Regex": "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "eval_loss", "Regex": "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "eval_accuracy", "Regex": "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "eval_f1", "Regex": "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "eval_precision", "Regex": "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "eval_recall", "Regex": "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "eval_runtime", "Regex": "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?" },
#     { "Name": "eval_samples_per_second", "Regex": "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?" },
]

estimator = HuggingFace(
    base_job_name = job_name,
    role = role,
    py_version = "py36",
    pytorch_version = "1.6.0",
    transformers_version = TRANSFORMERS_VERSION,
    entry_point = "trainer.py",
    instance_count = 1,
    instance_type = "ml.p3.16xlarge",
    source_dir = "./scripts",
    enable_sagemaker_metrics = True,
    metric_definitions = metric_definitions,
    hyperparameters = {
        "epochs": 3,
        "eval_batch_size": 128,
        "model_name": model_name,
        "train_batch_size": 64
    }
)

<span style="font-size: 16px;"><a href="https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs/">See Training Jobs in the SageMaker Console</a></span>

### Train a Model using the Estimator
##### ⏰ About 15 minutes

In [14]:
%%time

inputs = {
    "train": training_input_path, 
    "test": test_input_path
}
estimator.fit(inputs)

2021-05-13 21:37:42 Starting - Starting the training job...
2021-05-13 21:38:06 Starting - Launching requested ML instancesProfilerReport-1620941862: InProgress
............
2021-05-13 21:40:06 Starting - Preparing the instances for training.........
2021-05-13 21:41:27 Downloading - Downloading input data...
2021-05-13 21:42:07 Training - Downloading the training image..............bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-13 21:44:19,641 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-05-13 21:44:19,720 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-13 21:44:19,728 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-13 21:44:20,117 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Atte

### How'd It Go?

In [ ]:
from sagemaker import TrainingJobAnalytics
df = TrainingJobAnalytics(training_job_name = estimator.latest_training_job.name).dataframe()
df = df[["metric_name", "value"]]

summary = df.groupby("metric_name").describe()
summary.columns = summary.columns.droplevel(0)
summary = summary.reset_index().rename(columns = { 
    "metric_name": "Metric",
    "min": "Min", 
    "max": "Max", 
    "mean": "Average" 
}).set_index("Metric")
summary = summary.drop(["std", "count", "25%", "50%", "75%"], axis = 1).drop(["epoch"])
display(summary)

---
# Model Deployment

### Prepare the Model for Deployment

In [ ]:
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

class SentimentAnalysis(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(
            endpoint_name, 
            sagemaker_session = sagemaker_session, 
            serializer = JSONSerializer(), 
            deserializer = JSONDeserializer()
        )

name = name_from_base(job_name)

model = PyTorchModel(
    name = name,
    role = role, 
    model_data = estimator.model_data,
    source_dir = "./scripts",
    entry_point = "predictor.py",
    framework_version = "1.6.0",
    py_version = "py36",
    predictor_cls = SentimentAnalysis
)

In [ ]:
%%time

predictor = model.deploy(
    initial_instance_count = 1,
    instance_type = "ml.m5.large",
    endpoint_name = name,
    wait = True
)

### Make Inferences Using a SageMaker Predictor

In [ ]:
import json

inputs = [
    "Willow is the greatest movie that ever lived.",
    "The Notebook is ironically depressing.",
    "It's annoying that I had to Google the capitalization of \"Back to the Future\", but it is a gem of nostalgic wonder.",
    "Yikes! Weird Science did not age well for 2021."
]

for it in inputs:
    prediction = predictor.predict({"text": it})
    print(f'    {prediction}: {it}')

---
# Clean Up

In [ ]:
try:
    predictor.delete_endpoint()
    model.delete_model()
except:
    display("Already deleted")